<a href="https://colab.research.google.com/github/PosgradoMNA/actividades-del-projecto-equipo-5/blob/main/ActividadSemanal4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TC4029 Ciencia y analítica de datos
##Actividad Semanal 4
###Profesor Jobish Vallikavungal

**EQUIPO 5:**

* Laura Irán González Ojeda
 * Matrícula: A01794099

* Marcela Alejandra Rosales Jiménez
 * Matrícula: A01032022

Fecha de entrega: 11 de Octubre de 2022

**Objetivo:** Realizar la reducción de características de un conjunto de datos para encontrar sus componentes principales y mejorar su interpretabilidad con la menor pérdida de información posible.

## Importación del dataset

A continuación se realizará el proceso de importación del dataset, aplicando el procedimiento de selección y limpieza que se siguió en la práctica 3, ya que se estará utilizando el mismo universo de datos. 

Esto con el objetivo de tener un conjunto de datos normalizado y listo para su análisis.

In [1]:
# Importar librarias necesarias para el análisis de datos
import pandas as pd
import numpy as np
# Definir ruta de archivo CSV y exportar los datos del dataset
data_path = "https://raw.githubusercontent.com/PosgradoMNA/Actividades_Aprendizaje-/main/default%20of%20credit%20card%20clients.csv"
df = pd.read_csv(data_path)
df2 = df.dropna(subset=['X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23'])
# Transformacion de columna X3
mask = (df2['X3'] == 0) | (df2['X3'] == 5 ) | (df2['X3'] == 6 ) | (df2['X3'].isna())
df2.iloc[mask] = 4
# Transformacion de columna X4
mask = (df2['X4'] == 0) | (df2['X4'].isna())
df2.iloc[mask] = 3
# Transformacion de columna X5
mask = (df2['X5'].isna())
df2.iloc[mask] = -1
# Transformacion de columnas X6 a X11
col_historial_pagos = ['X6', 'X7', 'X8', 'X9', 'X10', 'X11']
for col in col_historial_pagos:
    # Transformar pay duly de -1 a 0
    mask = (df2[col] == -1)
    df2.iloc[mask] = 0
    # Transformar valores atipicos a -1
    mask = (df2[col] < 0) | (df2[col].isna())
    df2.iloc[mask] = -1

df2.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,ID,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,90000,2.0,2.0,2.0,34.0,0.0,0.0,0.0,0.0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0.0
3,4,50000,2.0,2.0,1.0,37.0,0.0,0.0,0.0,0.0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0.0
4,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#Parte 1: Ejercicio guiado 

Revise el ejercicio guiado para el análisis de componentes principales utilizando el conjunto de datos 

1. Paso 1: Determine el número mínimo de componentes principales que representan la mayor parte de la variación en sus datos

 * Utilice la proporción acumulada de la varianza que explican los componentes para determinar la cantidad de varianza que explican los componentes principales.

2. Paso 2: Interprete cada componente principal en términos de las variables originales

 * Examine la magnitud y la dirección de los coeficientes de las variables originales.
Nota: Cuanto mayor sea el valor absoluto del coeficiente, más importante será la variable correspondiente en el cálculo del componente.

3. Paso 3: Identifique valores atípicos

Realice alguna gráfica de valores atípicos o boxplot para identificar los valores atípicos. Cualquier punto que esté más alejado de la línea de referencia es un valor atípico.